# September 25

Application Programming Interfaces (APIs) allow us to interface with web-based services. For example, GenBank has an API that allows us to pull large amounts of genetic data from the web in an automated fashion. 

Today, we will not be using APIs, but we will do things the old fashioned way, to get a sense for why we want to use APIs. 

Here is a DNA sequence:

TACCACATCCAAGGAAGGCAGCAGGCGCGCAAA-TTACCCACTCCCGGCACGGGGAGGTAGTGACGAAAAATAACGATA-CGGGACTCATCCGAGGCCCCGTAATCGGAATGAGTACACTTTAAATCCTTTAACGAGGATCCATTGGAGGGCAAGTCTGGTGCCAGCAGCCGCGGTAATTCCAGCTCCAATAGCGTATATTAAAGTTGTTGCGGTTAAAAAGCTCG-TAGTTGAATCTGTGTC--GCACACCGTCGGTT--CACCGCT--CG--CGGT-GTTTA-ACTGGCGTGATGTGGGACGTCCTACCGG

Navigate to: https://www.ncbi.nlm.nih.gov/

On the right-hand side of the page, click on BLAST. Choose nucleotide BLAST:

![](img/blastn.png)


Paste in the nucleotide sequence, scroll to the bottom and click "BLAST". 

## This is just one example of an API, an interface that allows you to get data from the web.

Today, we will work with a number of APIs. The first is Open Tree of Life. We will download a phylogeny of salamanders from the database. Then, we will download nucleotide sequence data for the salamanders on the tree. We will build a phylogeny from the sequences, and plot the phylogeny over a map.

The base URL for all Open Tree inquiries is:

`https://api.opentreeoflife.org/v3/`

Open the [API reference](https://github.com/OpenTreeOfLife/germinator/wiki/Open-Tree-of-Life-Web-APIs). We will be extracting a subtree. Do you see a function on this page that says something about extracting subtrees?

In [ ]:
# Answer

# /tree_of_life/subtree

In [ ]:
! curl -X POST https://api.opentreeoflife.org/v3/tree_of_life/subtree -H "content-type:application/json" -d '{"node_id":"ott515349"}' > salamanders.json
    

Click on the function. What does the function do? I have navigated to the site for a [particular taxon](https://tree.opentreeoflife.org/opentree/opentree10.3@mrcaott57750ott128950/Plethodon-chlorobryonis--Plethodon-websteri). In Open Tree, every taxon has a unique identifier corresponding to where in the tree it is. The taxon will also be tagged to the paper from which its phylogenetic placement is derived.

Try to combine the base URL, function name, and node_id to make an API call. 

In [ ]:
! curl -X POST base_url_and_function_name_go_here -H "content-type:application/json" -d '{"node_id":"ott515349"}'


What's the problem with the above? The data aren't saved to a variable! Let's do it again, and save to a file.

In [ ]:
! curl -X POST https://api.opentreeoflife.org/v3/tree_of_life/subtree -d '{"node_id":"mrcaott57750ott128950", "output":"newick"}' > salamanders.json

This type of file is called a "JSON" file. It is a web data storage file meant for storing and transmitting data over the web. It is fairly popular, and so many libraries can read and write it. 

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 1000000
salamanders = pd.read_json("salamanders.json")
salamanders

What is in the newick column is actually a phylogeny! What we have downloaded is a mini-database enclosing the tree of the salamanders, and the reference study from which that tree was derived. We could even view the tree if we wanted.

In [ ]:
import dendropy
pd.options.display.max_colwidth = 1000000
newick_tree = salamanders.newick.to_string(index=False)
#View our Tree
newick_tree


In [ ]:
converted_tree = dendropy.Tree.get_from_string(newick_tree, schema = "newick")
converted_tree.as_string(schema="newick")

The tree is now a dendropy object, held in memory as a tree rather than a database or a string. Trees have some nice properties. For example, we could take a look at what the most recent common ancestor of two tips is:

In [ ]:
converted_tree.mrca(taxon_labels = ["Plethodon jordani ott515340", "Plethodon yonahlossee ott515343"])


What we're going to do is extract the names of the taxa from the list, and remove the OTT numbers. Then, we will put them in a list and see which ones have GenBank data. In the below cell, see if you can write a loop that will split each taxon name and remove the "OTTXXXX" part. On what character will you split it? 

In [ ]:
taxon_list = converted_tree.taxon_namespace.labels()

query_list = []

In [ ]:
taxon_list = converted_tree.taxon_namespace.labels()

query_list = []

#Answer is below

for taxon in taxon_list:
    taxon_split = taxon.split(" ")
    new_taxon = taxon_split[0] + "_" + taxon_split[1]
    query_list.append(new_taxon)

In [ ]:
len(query_list)

But we probably don't simply want all the data in GenBank for all 35 species - it could be tons of genes! So let's do a little scavenget hunt. We're going to find out what genes the study used, and pull those from GenBank. First, take a peek at the Open Tree documentation for its [bibliography functions](https://github.com/OpenTreeOfLife/germinator/wiki/Studies-API-v3#properties). Do you see one which might tell us how to access the name of the paper?

In [ ]:
# Answer
! curl -X POST https://api.opentreeoflife.org/v3/studies/find_studies \
-H "content-type:application/json" -d \
'{"property":"ot:studyId","value":"pg_423","verbose":true}'

To save us a little time, I've placed a copy of the paper in the Papers folder. Have a look - how many genes did they use to make this phylogeny? Scroll to the very bottom of the paper and find the supplemental data. Click on the link. This format is a little odd - it's just the alignment and the partition file, telling you which site belongs to which gene. Ideally, this data package would also contain information on which GenBank accessions the specimens go to, so that they could be verified. This should also include the unprocessed data, before multiple sequence alignment, so this step could be verified as well.

Because these sequences are aligned, we could have some trouble getting back good BLAST searches. But we're going to try. We will use dendropy to obtain the genetic data. Now, we will load the data. Our data are stored in the `data` directory. They are in the `phylip` schema. Take a peek at the [reference](https://dendropy.org/primer/reading_and_writing.html). We will use the function `dendropy.DNACharacterMatrix.get()`. See if you can assemble the command to read in the data. IT WILL TAKE A MOMENT TO READ IN.

In [ ]:
# Answer follows
amphib = dendropy.DnaCharacterMatrix.get(
    path="../data/plethodon.phy",
    schema="phylip"
)

Verify the length of the sequence, and compare it to the plethodon.phy file in data. 

In [ ]:
amphib.values()

Try tab completing. What are some of the methods available for these data? Particularly, have a look at `taxon_namespace` and `values`.

We have a phylogeny. We have data. First, let's estimate a quick and dirty tree.

In [ ]:
from dendropy.interop import raxml
rx = raxml.RaxmlRunner(raxml_path="/bin/raxmlHPC")
tree = rx.estimate_tree(
        char_matrix=amphib)

In [ ]:
tree.as_string(schema="newick")

It's a little odd that only part of the tree was uploaded. Pick a taxon that is present on the big tree, but not the smaller tree. Write it down. Mine is *Plethodon_cinereous*. Verify that it is not in the query list:

In [ ]:
amphib.taxon_namespace.labels

In [ ]:
# Answer follows

for taxon in query_list:
    if taxon == "Plethodon_kentucki":
        print("Pick a different taxon!")

Next, we will query the sequence out of the dataset. A phylogenetic dataset is basically a big dictionary in which the taxon name is a key, and the sequence is a value. Reminder: values can be retrieved from a dictionary like so:

`value = dictionary[key]`

Try it in the cell below.

In [ ]:
# Answer follows

my_taxon_sequence = amphib["Plethodon_kentucki"].symbols_as_string()
print(my_taxon_sequence)

So now we have one sequence. But it's made up of 12 genes. We're going to try to BLAST one gene. In the data folder, there is a file called `amphibians.models`. Open it up. Pick your favorite gene. Mine is `POMC_1` 

In [ ]:
cytb1 = my_taxon_sequence[4097:5236]

In [ ]:
cytb1


We're getting there - we have a taxon; we have one gene. Let's see if it matches anything in NCBI. The below command is roughly what we will enter. 

In [ ]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO

record = SeqIO.read("../data/sequence.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))

We need to have our sequences stored in a fasta format. We will now generate a fasta-formatted sequence file to query.

In [ ]:
fasta_pleth = {"Plethodon_cinereus": POMC_1}
ofile = open("../data/sequence.fasta", "w")

for item in fasta_pleth.keys():
    ofile.write(">" + item + "\n" + fasta_pleth[item] + "\n")

ofile.close()


Now that we have a fasta file, let's try to query GenBank.

In [ ]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO

record = SeqIO.read("../data/sequence.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))

Oh no, error! Let's process this one together, and come up with a fix.

For Thurs, we now have a BLAST hit. Please have a look at [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3820096/#!po=0.833333), on interpreting BLAST output.

For Thursday: AMW - add node annotation how-to.